# Recommendation Data Pipeline

In [ ]:
{
  "type": "DelimitedExtract",
  "name": "load ratings extract",
  "environments": [
    "production",
    "test"
  ],
  "inputURI": "file:///data/ml-latest/ratings.csv",
  "outputView": "ratings_raw",
  "header": true
}

In [ ]:
{
  "type": "TypingTransform",
  "name": "apply typing to the raw ratings dataset",
  "environments": ["production", "test"],
  "schemaURI": "file:///home/jovyan/src/ratings.json",
  "inputView": "ratings_raw",            
  "outputView": "ratings",
  "persist": true  
}

In [ ]:
{
  "type": "DelimitedExtract",
  "name": "load movies extract",
  "environments": [
    "production",
    "test"
  ],
  "inputURI": "file:///data/ml-latest/movies.csv",
  "outputView": "movies_raw",
  "header": true
}

In [ ]:
{
  "type": "TypingTransform",
  "name": "apply typing to the raw movie dataset",
  "environments": ["production", "test"],
  "schemaURI": "file:///home/jovyan/src/movies.json",
  "inputView": "movies_raw",            
  "outputView": "movies",
  "persist": true  
}

In [ ]:
{
  "type": "MLTransform",
  "name": "apply machine learning model to generate recommendations",
  "environments": [
    "production",
    "test"
  ],
  "inputURI": "file:///model/movie_lens_als",
  "inputView": "ratings",
  "outputView": "customer_ratings"
}

In [ ]:
%sql outputView=userRecommendations
SELECT userId, collect_list(title) as top_10 FROM 
   (SELECT userId, movieId, dense_rank() 
         OVER (PARTITION BY userId ORDER BY prediction DESC) as rank 
    FROM customer_ratings) tmp
JOIN movies ON tmp.movieId = movies.movieId
WHERE rank <= 10
GROUP BY userId